In [1]:
from IPython.display import HTML, display
import matplotlib.pyplot as plt
import numpy as np
import folium

from irsdk import IRDiskClient
from irsdk.util.logging import info, debug

from irsdk.models import *



Models path = Y:\code\sim-racing\irsdk-interop\packages\python\irsdk\models


In [2]:
lt_file = "C:/Users/jglanz/AppData/Local/VRKit/TrackMaps/448__indianapolis_2022_road__Road_Course.trackmap"
lt_file_wrapper = open(lt_file, "rb")
lt_data = lt_file_wrapper.read()
lt_file_wrapper.close();
lt = LapTrajectory()
lt = lt.FromString(lt_data)


# info(f"record_count={ir.record_count}")
# speed_pairs = ["Speed", list(filter(lambda v: v is not None and v != 0.0, ir.get_all("Speed")))]


lats = [] #list(filter(lambda v: v is not None and v != 0.0, ir.get_all("Lat")))
lons = [] #list(filter(lambda v: v is not None and v != 0.0, ir.get_all("Lon")))
for point in lt.path:
  lats.append(point.latitude)
  lons.append(point.longitude)

lats = list(filter(lambda v: v is not None and v != 0.0, lats))
lons = list(filter(lambda v: v is not None and v != 0.0, lons))

coords = list(zip(lats, lons))

lat_center = (max(lats) + min(lats)) / 2
lon_center = (max(lons) + min(lons)) / 2

lat_pairs = ["Latitude", lats]
lon_pairs = ["Longitude", lons]
# 
# for [label, values] in [lat_pairs, lon_pairs, speed_pairs]:
#     info(label)
#     info(f"count={len(values)},max={max(values)},max={min(values)},avg={sum(values) / len(values)}`")
#     info(" ")

# Create a map centered on Iran
lt_map = folium.Map(location=[lat_center,lon_center], zoom_start=16)

# Loop through the data and add markers for each location
# for i, row in coords:
#     folium.Marker(
#         [row['Latitude'], row['Longitude']],
#         popup=row['Location']
#     ).add_to(session_map)

# Add a line to connect the markers
#locations = coords  # data[['Latitude', 'Longitude']].values.tolist()
folium.PolyLine(coords, color='red').add_to(lt_map)
# session_map_html = session_map._repr_html_()
# ir.close()
lt_map


In [4]:
from functools import reduce
ibt_file = "Y:\\code\\sim-racing\\irsdk-interop\\data\\ibt\\ibt-fixture-superformulalights324_montreal.ibt"
# ibt_file = "Y:\\data\\local-share\\iracing\\telemetry\\ligierjsp320_twinring fullrc 2023-12-24 22-43-26.ibt"
# ir = irsdk.IRSDK()
# ir.startup(test_file=ibt_file)
# ir.freeze_var_buffer_latest()

ir = IRDiskClient()
ir.open(ibt_file)

# info(f"record_count={ir.record_count}")
speed_pairs = ["Speed", list(filter(lambda v: v is not None and v != 0.0, ir.get_all("Speed")))]

lats = list(filter(lambda v: v is not None and v != 0.0, ir.get_all("Lat")))
lons = list(filter(lambda v: v is not None and v != 0.0, ir.get_all("Lon")))
laps = list(filter(lambda v: v is not None and v != 0, ir.get_all("Lap")))

coords = list(zip(lats, lons))

lat_center = (max(lats) + min(lats)) / 2
lon_center = (max(lons) + min(lons)) / 2

lat_pairs = ["Latitude", lats]
lon_pairs = ["Longitude", lons]
# 
# for [label, values] in [lat_pairs, lon_pairs, speed_pairs]:
#     info(label)
#     info(f"count={len(values)},max={max(values)},max={min(values)},avg={sum(values) / len(values)}`")
#     info(" ")

# Create a map centered on Iran
session_map = folium.Map(location=[lat_center,lon_center], zoom_start=16)

# Loop through the data and add markers for each location
# for i, row in coords:
#     folium.Marker(
#         [row['Latitude'], row['Longitude']],
#         popup=row['Location']
#     ).add_to(session_map)

# Add a line to connect the markers
#locations = coords  # data[['Latitude', 'Longitude']].values.tolist()
folium.PolyLine(coords, color='red').add_to(session_map)
# session_map_html = session_map._repr_html_()
# ir.close()
session_map

In [5]:
# display(HTML(session_map_html))

In [23]:

# ir = IBT()
# ir.open(ibt_file)
# 
# # info(f"record_count={ir.record_count}")
# speeds = []
# times = []
# for idx in range(0,ir.record_count):
# 	speed = ir.get(idx, "Speed")
# 	if speed is not None and speed > 0.0:
# 		times.append(ir.get(idx, "SessionTime"))
# 		speeds.append(speed)


In [ ]:
# 		
# plt.plot(times, speeds)
# plt.show()		


all_lap_pos_updates = list(zip(laps, lats, lons))

lap_oks = list(filter(lambda ok: ok is not None, ir.get_all("LapDeltaToBestLap_OK")))

def to_bad_laps(bad_laps, pair): 
	lap, ok = pair
	if ok or lap in bad_laps:
		return bad_laps
	
	bad_laps.append(lap)
	return bad_laps
	
bad_laps = reduce(to_bad_laps,list(zip(laps,lap_oks)), list())


def to_lap_pos_map(pos_map, data:[bool,int,float,float]):
	if pos_map is None:
		pos_map = dict()
	lap,lat,lon = data
	if lap not in pos_map:
		pos_map[lap] = list()
	
	positions = pos_map[lap]
	positions.append([lat,lon])
	return pos_map

laps_pos_map = reduce(to_lap_pos_map, all_lap_pos_updates,dict())

for lap in bad_laps:
	laps_pos_map[lap] = None 

laps_pos_map

# clean_lap_counts = [f"{data[0]}_{data[1]}" for data in list(zip(laps, ir.get_all("LapLasNLapSeq")))]
clean_lap_counts = [f"{data[0]}_{data[1]}" for data in list(zip(laps, ir.get_all('PlayerCarMyIncidentCount')))]

def unique(list1):

    # initialize a null list
    unique_list = []

    # traverse for all elements
    for x in list1:
        # check if exists in unique_list or not
        if x not in unique_list:
            unique_list.append(x)

    return unique_list

clean_lap_counts = unique(clean_lap_counts)
    
 
 

In [15]:
off_tracks = list(filter(lambda ok: ok is False, ir.get_all("LapDeltaToBestLap_OK")))